## Семинар 7. Сравнение двух выборок.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as st

### Парные выборки

#### Задача

Было проведено исследование, чтобы выяснить, повлияют ли новые диетические медикаменты на женщин, желающих сбросить вес. Вес 100 пациенток был измерен до лечения и через 6 недель ежедневного применения лечения. Данные приведены в файле "Weight.txt". При уровне значимости 5% можно ли сделать вывод, что лечение уменьшает вес?

In [2]:
data = pd.read_csv("Weight.txt")
x = data['x']
y = data['y']

In [3]:
st.shapiro(x)

(0.9929246306419373, 0.8834702372550964)

In [4]:
st.shapiro(y)

(0.9839687943458557, 0.2669937014579773)

#### t-test для парных выборок

In [5]:
#t-test для двух парных выборок:
from scipy.stats import ttest_rel
ttest_rel(x, y)
#второе значение результата - это p-value для двусторонней альтернативы

Ttest_relResult(statistic=13.848458612321073, pvalue=6.741025162804584e-25)

In [6]:
#если данные корректны, то для левосторонней альтернативы 
#необходимо просто поделить это p-value на 2:
t_test_res = ttest_rel(x, y)
t_test_res.pvalue/2.0

3.370512581402292e-25

In [7]:
#Или можем написать код вручную

#Создаем столбец из разностей значений y и x
z = y - x
#Применяем t-test для одной выборки
from scipy.stats import t
n = len(z)
z_mean = np.mean(z)
z_s = np.std(z, ddof=1)
t_stat = (z_mean - 0) * np.sqrt(n) / z_s # t - статистика для проверки гипотезы
t_p_value = t.cdf(t_stat, n-1) #левосторонняя альтернатива!
t_p_value

3.3705125814023646e-25

#### Критерий знаков

In [8]:
z = z[z!=0] #убираем для критерия знаков нулевые элементы Z_i
b = sum(z > 0) #количество Z_i > 0
n = len(z)

#Критерий знаков = бином.критерий с H_0: p=0.5 
from scipy.stats import binom_test
binom_test(b, n,  p=0.5)
#так получим p-value для двусторонней альтернативы

5.521266925181344e-17

In [9]:
#если данные корректны, для односторонней альтернативы необходимо просто поделить это p-value на 2:
sign_res=binom_test(b, n,  p=0.5)
sign_res/2.0

2.760633462590672e-17

In [10]:
#или можно посчитать вручную p-value с помощью функции распределения:
from scipy.stats import binom
binom.cdf(b, n, 0.5)

2.760633462590672e-17

Асимптотический критерий знаков

In [11]:
#Находим новую статистику:
b_star = (b - n*0.5) / np.sqrt(n*0.25)
#Находим новое p-value (для левосторонней альтернативы!):
from scipy.stats import norm
norm.cdf(b_star, loc=0, scale=1)

1.0125478785948885e-15

#### Знако-ранговый критерий Вилкоксона (Wilcoxon signed-rank test)

In [12]:
from scipy.stats import wilcoxon
#тест использует нормальную аппроксимацию (которую рекомендуется применять при n>20)
wilcoxon(x, y)
#второе значение результата - это p-value для двусторонней альтернативы

WilcoxonResult(statistic=104.0, pvalue=1.2731454873548327e-16)

In [13]:
#если данные корректны, для односторонней альтернативы необходимо просто поделить это p-value на 2:
signed_rank_res = wilcoxon(x, y)
signed_rank_res.pvalue/2.0

6.365727436774163e-17

### Независимые выборки

#### Задача

Для сравнения уровня заработной платы были отобраны в соответствии со стажем работники-мужчины и работники-женщины. В файлах "Male.txt" и "Female.txt" содержатся получившиеся данные (в тысячах рублей). Можно ли утверждать на уровне значимости 5%, что зарплата женщин ниже? 

In [9]:
data1 = pd.read_csv("Male.txt")
male = data1['male']
data2 = pd.read_csv("Female.txt")
female = data2['female']

In [15]:
st.shapiro(male)

(0.9931766986846924, 0.48368388414382935)

In [16]:
st.shapiro(female)

(0.9916744232177734, 0.5287119746208191)

#### t-test для независимых выборок

Сначала мы должны проверить равенство дисперсий двух выборок

In [17]:
from scipy.stats import f #распределение Фишера
def F_test(x, y):
    x = np.array(x)
    y = np.array(y)
    df1 = len(x) - 1
    df2 = len(y) - 1
    F_stat = np.var(x, ddof=1)/np.var(y, ddof=1)
    pv = 2*np.min([f.cdf(F_stat, df1, df2), 1 - f.cdf(F_stat, df1, df2)])
    return pv

In [18]:
F_test(male, female)

0.0008331396314132209

Дисперсии получились не равны, поэтому для проверки равенства средних
используем t-test с параметром equal_var=False !!

In [19]:
from scipy.stats import ttest_ind
ttest_ind(male, female, equal_var=False)
#второе значение результата - это p-value для двусторонней альтернативы

Ttest_indResult(statistic=1.1617783123619438, pvalue=0.2463428597840339)

In [20]:
#если данные корректны, то для односторонней альтернативы 
#необходимо просто поделить это p-value на 2:
t_res = ttest_ind(male, female, equal_var=False)
t_res.pvalue/2.0

0.12317142989201695

#### Критерий Манна - Уитни (Mann–Whitney U-test)

In [10]:
from scipy.stats import mannwhitneyu
mannwhitneyu(female, male, alternative='less') #альтернатива: female is less than male

MannwhitneyuResult(statistic=13884.0, pvalue=0.1168621859831947)

In [11]:
#только p-value:
res = mannwhitneyu(female, male, alternative='less')
res.pvalue

0.1168621859831947

In [5]:
mannwhitneyu?

### Box–Cox transformation

In [23]:
from scipy.stats import gamma
x = gamma.rvs(size=50, a = 2, scale = 1) 
y = gamma.rvs(size=40, a = 2.4, scale = 1)

In [24]:
mannwhitneyu(x, y, alternative='two-sided')

MannwhitneyuResult(statistic=810.0, pvalue=0.12386873494301738)

In [25]:
st.shapiro(x)

(0.8965877294540405, 0.0003714040503837168)

In [26]:
st.shapiro(y)

(0.9202797412872314, 0.00785873644053936)

Гипотеза о нормальности для обеих выборок отвергается

In [27]:
st.boxcox(x)[0] #новые данные после преобразования

array([ 0.81158716, -0.65003146,  0.46595561,  0.27961057,  0.40191741,
        1.38600589, -0.80027568, -0.81013103,  2.04307747, -0.02333145,
        0.25310435,  0.5260058 ,  1.24350564, -0.766802  ,  0.33855418,
        1.81952994,  1.16276781,  0.05475102, -0.29551425,  0.7977478 ,
        0.79781808,  1.44974071,  0.49258208,  0.64168527,  0.00823905,
        2.23151824,  0.76697799,  0.42722777,  0.69436378,  0.12806351,
        0.56113453,  0.92544032, -1.2503075 ,  0.82278375, -0.38266837,
        0.65853667, -0.26838935,  1.73359341,  0.29750066,  1.33840871,
       -0.57200377,  0.304161  ,  0.54634276,  0.41620634,  2.03019687,
        1.39278733,  0.81340695,  1.07696808, -0.03397915,  1.15594171])

In [28]:
st.boxcox(x)[1] #параметр лямбда

0.2533408375169048

In [29]:
x1 = st.boxcox(x)[0]
st.shapiro(x1)

(0.9864102005958557, 0.8301360011100769)

In [30]:
st.boxcox(y)[1]

0.3100060583320573

In [31]:
y1 = st.boxcox(y)[0]
st.shapiro(y1)

(0.9773542881011963, 0.5921720862388611)

In [32]:
#параметры лямбда отличаются, можем для преобразования y взять лямбду из преобразования x
l_x = st.boxcox(x)[1]
y2 = st.boxcox(y, lmbda = l_x)
st.shapiro(y2)

(0.9762006998062134, 0.5511516332626343)

##### Сомнительный трюк

In [33]:
F_test(x1, y2)

0.5042783476225778

In [34]:
ttest_ind(x1, y2, equal_var=True)

Ttest_indResult(statistic=-1.6550049562112945, pvalue=0.10148588411950796)

### Сравнение распределений двух выборок (проверка на однородность)

(только для непрерывных распределений)

In [35]:
from scipy.stats import norm, t
x = norm.rvs(size = 200, loc = 0, scale = 1)
y = t.rvs(size=300, df = 7)
z = norm.rvs(size = 400, loc = 0, scale = 3)

#### Критерий Смирнова

In [36]:
from scipy.stats import ks_2samp
ks_2samp(x, y)

Ks_2sampResult(statistic=0.07833333333333334, pvalue=0.43796584564429464)

In [37]:
ks_2samp(x, z)

Ks_2sampResult(statistic=0.3175, pvalue=2.544853217045784e-12)

#### Общий критерий Андерсона-Дарлинга

In [38]:
from scipy.stats import anderson_ksamp
anderson_ksamp([x, y])

Anderson_ksampResult(statistic=1.3319114094966367, critical_values=array([0.325, 1.226, 1.961, 2.718, 3.752, 4.592, 6.546]), significance_level=0.09178048892010211)

В качестве результата тест выдает значение статистики, набор квантилей 
$x_{1-\alpha}$ для значений $\alpha$ вида 25%, 10%, 5%, 2.5%, 1%, 0.5%, 0.1% и p-value.

In [39]:
#С помощью общего критерия Андерсона-Дарлинга можно проверять на однородность больше, чем две выборки
anderson_ksamp([x, y, z])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: p-value floored: true value smaller than 0.001
  


Anderson_ksampResult(statistic=41.394015370249406, critical_values=array([0.44925884, 1.3052767 , 1.9434184 , 2.57696569, 3.41634856,
       4.07210043, 5.56419101]), significance_level=0.001)

In [40]:
anderson_ksamp([x, z])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: p-value floored: true value smaller than 0.001
  """Entry point for launching an IPython kernel.


Anderson_ksampResult(statistic=36.39134200117951, critical_values=array([0.325, 1.226, 1.961, 2.718, 3.752, 4.592, 6.546]), significance_level=0.001)